<a href="https://colab.research.google.com/github/miladmirzazadeh/Gamein-Prototype/blob/master/prototype_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



TESTING CODES ############################################################################



In [80]:
Controller.create_player("raman")
Controller.create_player("ali") 
raman = Controller.player_objects["raman"]
raman_stockhouse = list(raman.land_objects.values())[0].stockhouse

ali = Controller.player_objects["ali"]
ali_stockhouse = list(ali.land_objects.values())[0].stockhouse


In [89]:
Controller.start_production("ali","tomato", 1110)

yesss


In [90]:
ali_stockhouse.inventory_df

,total_inventory,inhand_inventory
potato,100,100
tomato,1110,1110


In [91]:
Controller.put_into_market("ali", "tomato", 500, 200)

'Done :)'

In [92]:
Market.vitrin

,seller_obj,product_name,stockhouse_obj,amount,price
40,<__main__.Player object at 0x7f2750a91d30>,potato,<__main__.StockHouse object at 0x7f2750a91f28>,400,200
506,<__main__.Player object at 0x7f2750a91908>,tomato,<__main__.StockHouse object at 0x7f2750a27b38>,500,200


In [85]:
Controller.buy_product("ali",Market.vitrin.index[0],100)

'Done :)'

In [93]:
Controller.show_product_market_info()

,id,product_name,amount,price,seller_name
40,40,potato,400,200,raman
506,506,tomato,500,200,ali


END   TESTING CODES ######################################################################









In [1]:
import numpy as np 
import pandas as pd 
import random

In [79]:
class Controller():

  #init values 
  Market.vitrin = pd.DataFrame([], columns=["seller_obj","product_name", "stockhouse_obj","amount", "price"])
  ##############################
  return_message=""
  turn = 0 
  last_turn = 50
  player_objects = dict()#name,object

  def next_turn():
    turn+=1
    if turn == Controller.last_turn : 
      score_board = Controller.score_board()
      Controller.return_message = "End Game. Thank u " + score_board

  
  def score_board():
    net_worths = pd.Series([])
    for player in list(player_objects.keys()):
      obj = player_objects[player]
      obj.calculate_final_net_worth()
      net_worths[player] = obj 
    net_worths = net_worths.sort(ascending=False)
    return net_worths


  # def get_status(player_name):
    # for land in player_objecs[player_name].

  def create_player(player_name):
    land_id = random.choice(Land.available_lands) #random از بین لیست زمین های در دسترس
    Land.available_lands.remove(land_id)   #remove from available lands
    machine_id =  2 # از دیتافریم زمین و ماشین خوانده میشود
    Controller.player_objects.update({player_name : Player(player_name)})
    Controller.buy_land(player_name, land_id)
    Controller.buy_machine(player_name, land_id, machine_id)
    
  def buy_land(player_name, land_id):
    Controller.player_objects[player_name].add_land(land_id)
    # Controller.player_objects[name].cash -=         اینجا باید پول زمین رو از حسابش کم کنیم . فعلا لازم نیست  چون خودمون بهش زمین رو اختصاص میدیم 

  def buy_machine(player_name, land_id, machine_id):
    Controller.player_objects[player_name].add_machine(land_id, machine_id)

  def start_production(player_name, product_name, amount):# This Func should take machine_id too ( Later Inshallah)
    if not (product_name in Product.products_list):
      return ("Do we have this Product in the Game ?!")
    else : 
      player_obj = Controller.player_objects[player_name]
      stockhouse = list(player_obj.land_objects.values())[0].stockhouse
      if stockhouse.remaining_cap <= amount : 
        return ("No Enough Space")
      else :
        print("yesss")
        stockhouse.add(product_name, amount)
      


  def put_into_market(player_name, product_name, amount, price):# it doesn't get stockhouse_id or etc. until we have more than one stockhouse
    product_on_vitrin_id = random.choice([i for i in range(1000) if not(i in Market.vitrin.index)])   # a random unique id , that will assign to vitrin objects
    player_obj = Controller.player_objects[player_name]
    stockhouse_obj = list(player_obj.land_objects.values())[0].stockhouse
    

    #check the inventory
    if not (product_name in stockhouse_obj.inventory_df.index):
      return ("Doesn't Exist")
    else:
      if stockhouse_obj.inventory_df.loc[product_name, "inhand_inventory"] < amount : 
         return("No Enough Inventory")
      else :
        Market.vitrin.loc[product_on_vitrin_id] = [player_obj, product_name, stockhouse_obj, amount, price]
        stockhouse_obj.inventory_df.loc[product_name, "inhand_inventory"] -= amount
        return ("Done :)")
    
  
  def upgrade_stockhouse(player_name, land_id):
    pass

  def show_product_market_info():
    vitrin = Market.vitrin.copy()
    vitrin["seller_name"] = [obj.name for obj in vitrin.seller_obj]
    vitrin["id"] = vitrin.index
    vitrin = vitrin[["id", "product_name" , "amount", "price", "seller_name"]]
    return vitrin

  def buy_product(player_name, product_on_vitrin_id, amount): # آیدی محصول داخل ویترین رو میده که بریم از روی دیکشنری توی مارکت آبجکتش رو پیدا کنیم و انتقال رو انجام بدیم 
    buyer_player = Controller.player_objects[player_name]
    buyer_stockhouse = list(buyer_player.land_objects.values())[0].stockhouse
    product_on_vitrin_obj = Market.vitrin.loc[product_on_vitrin_id]
    seller_player = Market.vitrin.loc[product_on_vitrin_id, "seller_obj"]
    
    if amount > buyer_stockhouse.remaining_cap:
      return ("No Enough Space")
    elif buyer_player.cash < product_on_vitrin_obj.price * amount :
      return ("No Enough Cash")
    elif amount > product_on_vitrin_obj.amount :
      return ("No Enough Product In Vitrin")
    else:
      buyer_player.cash -= amount * product_on_vitrin_obj.price #update cash
      buyer_stockhouse.add(product_on_vitrin_obj.product_name, amount) #updata buuyer inventory


      #update seller inventory :
      seller_stockhouse = list(seller_player.land_objects.values())[0].stockhouse
      seller_stockhouse.remove(product_on_vitrin_obj.product_name, amount)
      seller_player.cash += amount * product_on_vitrin_obj.price # add cash to seller player


      #update vitrin
      product_on_vitrin_obj.amount -= amount

      return ("Done :)")
    
   

In [4]:
class Player():
  players_name=[]

  def __init__(self, player_name):
    self.name = player_name
    #remove from available
    self.cash = 1000000 # بودجه اولیه
    self.land_objects=dict()

  def calculate_final_net_worth(self):
    inventory_value_list = []
    for land in list(self.land_objects.keys()):
      inventory_value_list.append(self.land_objects[land].calculate_inventory_value()) # تو هر خونه این لیست ، ارزش دارایی های یک زمین از زمین های بازیکن ثبت میشه
    inventory_value_sum = sum(inventory_value_list)
    net_worth = inventory_value_sum + self.cash 
    return net_worth 

  def add_land(self, land_id):
    self.land_objects.update({land_id : Land(land_id)}) # یه اینستنس از لند میسازه که ورودی بهش آیدی میدیم . بعد توی خود لند ، بر حسب آیدی بقیه فیچر های لند رو ست میکنیم
    

  def add_machine(self, land_id, machine_id):
    self.land_objects[land_id].add_machine(machine_id)







In [5]:
class Gamein_Bazaar():

  def put_into_market():
    pass

In [9]:
class Land():
  lands_list =[] #amirreza یه دیتا فریم که یه آی دی زمین داره و ماشین
  available_lands= list(range(100)) #آی دی های باقی مانده
  
  def __init__(self, land_id):
    self.machine_objects = dict()
    self.land_id = land_id
    self.stockhouse = StockHouse()# اینجا دیگه دیکشنری نیست چون فرض بر اینه که یه دونه انبار بیشتر نداریم 


    # self.stockhouse = StockHouse()
  def add_machine(self, machine_id):
    self.machine_objects.update({machine_id: Machine(machine_id)})

    


  

In [10]:

class StockHouse():
  def __init__(self):
    self.total_cap = 3000
    self.remaining_cap = self.total_cap
    self.inventory_df = pd.DataFrame([], columns= ["total_inventory", "inhand_inventory"]) # indexes = product names 
    self.cost_per_product = 100
  def add(self, product_name, amount):
    if self.remaining_cap <= amount : 
      return ("No Enough Space")
    else :
      if product_name in self.inventory_df.index:
        self.inventory_df.loc[product_name, ["total_inventory", "inhand_inventory"]] += amount
      else:
        self.inventory_df.loc[product_name, ["total_inventory", "inhand_inventory"]] = amount 
      self.remaining_cap = self.total_cap - self.inventory_df.total_inventory.sum()
      return (True)
  def remove(self, product_name, amount, just_total=True):
    if just_total : 
      self.inventory_df.loc[product_name, ["total_inventory"]] -= amount 
      self.remaining_cap += amount 


    



  

In [12]:
class Machine():
  machines_list = _

  def __init__(self,machine_id):
    self.production_id = _
    self.price = _ 
    self.enabled = True
    self.cost_per_production = _
    self.working = 0
    self.remaining_production = _
    
  def start_production(self):
    pass
  def stop_production(self):
    pass
  

  

In [16]:
class Product():
  products_list = ["corn","rice","tomato","potato","salad_olvie","gheyme"]
  def __init__(self):
    self.material_needed = 10
    self.lower_band = 1
    self.upper_band = 20

    
  

In [17]:
class Material():
  Prices = _ #dict (id , price)

In [18]:
class Market():
  vitrin = pd.DataFrame([], columns=["seller_obj","product_name", "stockhouse_obj","amount", "price"])


In [19]:
#####################################################
#################################################### DELETED


class ProductOnVitrin():
  def __init__(self, ):
    self.seller_obj = player_obj
    self.stockhouse_obj = stockhouse_obj
    self.product_obj = product_obj
    self.amount = amount
    self.price = price